In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install datasets transformers==4.11.2
!pip install sentencepiece
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 31.9 MB/s 
     |████████████████████████████████| 2.9 MB 62.1 MB/s 
     |████████████████████████████████| 182 kB 71.0 MB/s 
     |████████████████████████████████| 3.3 MB 57.4 MB/s 
     |████████████████████████████████| 880 kB 48.7 MB/s 
     |████████████████████████████████| 212 kB 69.5 MB/s 
     |████████████████████████████████| 132 kB 77.5 MB/s 
     |████████████████████████████████| 127 kB 76.2 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=19f4984de5153ec976cf9b3c11ad15ef5a7cc12664afbde36ad3c00a3b4b5fe5
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c15899bf659ba74f623ac776e861cf2eb8608c1825ddec66a4
Successfully built sacremoses
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstal

#Preprocess data

In [3]:
squad_v2 = False
model_checkpoint = "airesearch/wangchanberta-base-att-spm-uncased"
batch_size = 16

In [4]:
from datasets import load_dataset, load_metric

In [5]:
datasets = load_dataset("thaiqa_squad")

Generating train split:   0%|          | 0/4000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/74 [00:00<?, ? examples/s]

Dataset thaiqa_squad downloaded and prepared to /root/.cache/huggingface/datasets/thaiqa_squad/thaiqa_squad/1.0.0/fce14864b511d48464540780f328f4b415746b63f2fd934ad0b06c3eead7787b. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
datasets

DatasetDict({
    train: Dataset({
        features: ['question_id', 'article_id', 'context', 'question', 'answers'],
        num_rows: 4000
    })
    validation: Dataset({
        features: ['question_id', 'article_id', 'context', 'question', 'answers'],
        num_rows: 74
    })
})

In [7]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/282 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/546 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/883k [00:00<?, ?B/s]

In [8]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [9]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

In [10]:
for i, example in enumerate(datasets["train"]):
    if len(tokenizer(example["question"], example["context"])["input_ids"]) > 384:
        break
example = datasets["train"][i]

In [11]:
len(tokenizer(example["question"], example["context"])["input_ids"])

520

In [12]:
len(tokenizer(example["question"], example["context"], max_length=max_length, truncation="only_second")["input_ids"])

384

In [13]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    stride=doc_stride
)

In [14]:
[len(x) for x in tokenized_example["input_ids"]]

[384, 294]

In [15]:
for x in tokenized_example["input_ids"][:2]:
    print(tokenizer.decode(x))

<s> สุนัขตัวแรกรับบทเป็นเบนจี้ในภาพยนตร์เรื่อง <unk>enji ที่ออกฉายในปี พ.ศ. 2517 มีชื่อว่าอะไร</s></s> <doc id="115035" url="https://th.wikipedia.org/wiki?curid=115035" title="เบนจี้">เบนจี้ เบนจี้ () เป็นชื่อตัวละครหมาพันทางแสนรู้ ที่ปรากฏอยู่ในภาพยนตร์หลายเรื่องที่เขียนบท และกํากับโดย โจ แคมป์ ในช่วงทศวรรษ 1970 ถึง 1980 ภาพยนตร์เรื่องแรกในชุด ใช้ชื่อเรื่องว่า เบนจี้ เช่นเดียวกับตัวละคร ถ่ายทําที่เมืองดัลลัส รัฐเทกซัส ฉายครั้งแรกในปี พ.ศ. 2517 ภาพยนตร์ได้รับการเสนอชื่อเข้าชิงรางวัลออสการ์ และได้รางวัลลูกโลกทองคํา สาขาเพลงประกอบยอดเยี่ยม จากเพลง <unk>enji's <unk>heme (<unk> <unk>eel <unk>ove) ร้องโดย ชาร์ลี ริช หมาที่แสดงเป็นเบนจี้ตัวแรก ชื่อว่า ฮิกกิ้นส์ (พ.ศ. 2502 - พ.ศ. 2518) มีอายุถึง 15 ปีแล้วในขณะแสดง หลังจากภาพยนตร์ออกฉายได้ไม่นาน มันก็ตายในปี พ.ศ. 2518เบนจี้ในภาพยนตร์เบนจี้ในภาพยนตร์. - พ.ศ. 2517, <unk>enji (ภาพยนตร์) - พ.ศ. 2520, <unk>or the <unk>ove of <unk>enji (ภาพยนตร์) - พ.ศ. 2521, <unk>enji's <unk>ery <unk>wn <unk>hristmas <unk>tory (ภาพยนตร์โทรทัศน์) - พ.ศ. 2523, <unk>h

In [16]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    stride=doc_stride
)
print(tokenized_example["offset_mapping"][0][:100])

[(0, 0), (0, 1), (0, 5), (5, 11), (11, 20), (20, 23), (23, 26), (26, 36), (36, 42), (42, 43), (43, 44), (44, 46), (46, 48), (48, 49), (49, 52), (52, 58), (58, 62), (62, 63), (63, 64), (64, 65), (65, 66), (66, 67), (67, 68), (68, 72), (72, 73), (73, 82), (82, 86), (0, 0), (0, 0), (0, 1), (0, 1), (1, 3), (3, 4), (4, 5), (5, 7), (7, 8), (8, 9), (9, 10), (10, 13), (13, 15), (15, 16), (16, 17), (17, 19), (19, 20), (20, 21), (21, 22), (22, 27), (27, 30), (30, 32), (32, 33), (33, 42), (42, 43), (43, 46), (46, 47), (47, 51), (51, 52), (52, 55), (55, 57), (57, 59), (59, 62), (62, 64), (64, 65), (65, 66), (66, 71), (71, 72), (72, 73), (73, 76), (76, 79), (79, 80), (80, 81), (81, 84), (84, 87), (87, 88), (88, 91), (91, 94), (94, 96), (96, 97), (97, 98), (98, 106), (106, 113), (113, 116), (116, 119), (119, 122), (122, 125), (125, 128), (128, 129), (129, 132), (132, 141), (141, 151), (151, 161), (161, 164), (164, 171), (171, 175), (175, 183), (183, 184), (184, 186), (186, 187), (187, 192), (192, 19

In [17]:
first_token_id = tokenized_example["input_ids"][0][1]
offsets = tokenized_example["offset_mapping"][0][1]
print(tokenizer.convert_ids_to_tokens([first_token_id])[0], example["question"][offsets[0]:offsets[1]])

▁ ส


In [18]:
sequence_ids = tokenized_example.sequence_ids()
print(sequence_ids)

[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [19]:
answers = (example["answers"])
start_char = answers["answer_begin_position"][0]
end_char = start_char + len(answers["answer"][0])

# Start token index of the current span in the text.
token_start_index = 0
while sequence_ids[token_start_index] != 1:
    token_start_index += 1

# End token index of the current span in the text.
token_end_index = len(tokenized_example["input_ids"][0]) - 1
while sequence_ids[token_end_index] != 1:
    token_end_index -= 1

# Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
offsets = tokenized_example["offset_mapping"][0]
if (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
    # Move the token_start_index and token_end_index to the two ends of the answer.
    # Note: we could go after the last offset if the answer is the last word (edge case).
    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
        token_start_index += 1
    start_position = token_start_index - 1
    while offsets[token_end_index][1] >= end_char:
        token_end_index -= 1
    end_position = token_end_index + 1
    print(start_position, end_position)
else:
    print("The answer is not in this feature.")

198 201


In [20]:
print(tokenizer.decode(tokenized_example["input_ids"][0][start_position: end_position+1]))
print(answers["answer"][0])

ฮิกกิ้นส์
ฮิกกิ้นส์


In [21]:
pad_on_right = tokenizer.padding_side == "right"

In [22]:
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = (examples["answers"][sample_index])
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_begin_position"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_begin_position"][0]
            end_char = start_char + len(answers["answer"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [23]:
features = prepare_train_features(datasets['train'][:5])

In [24]:
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [25]:
def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["question_id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [26]:
validation_dataset = datasets["validation"].map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=datasets["validation"].column_names,
)
len(datasets["validation"]), len(validation_dataset)

  0%|          | 0/1 [00:00<?, ?ba/s]

(74, 320)

#Fine-tune Model

In [27]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/404M [00:00<?, ?B/s]

Some weights of the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased were not used when initializing CamembertForQuestionAnswering: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing CamembertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForQuestionAnswering were not initialized from the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased and are newly initialized: ['qa_outputs.bias', 

In [28]:
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-squad",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.01,
    save_steps=2000,
)

In [29]:
from transformers import default_data_collator

data_collator = default_data_collator

In [30]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
)

In [31]:
trainer.train()

***** Running training *****
  Num examples = 20118
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1258


Epoch,Training Loss,Validation Loss
1,1.106900,1.156017


***** Running Evaluation *****
  Num examples = 320
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1258, training_loss=1.3217516051566847, metrics={'train_runtime': 1415.5349, 'train_samples_per_second': 14.212, 'train_steps_per_second': 0.889, 'total_flos': 3942576114011136.0, 'train_loss': 1.3217516051566847, 'epoch': 1.0})

In [32]:
trainer.save_model("./model-QA-wangchanberta")

Saving model checkpoint to ./model-QA-wangchanberta
Configuration saved in ./model-QA-wangchanberta/config.json
Model weights saved in ./model-QA-wangchanberta/pytorch_model.bin


In [33]:
tokenizer.save_pretrained("./model-QA-wangchanberta")

tokenizer config file saved in ./model-QA-wangchanberta/tokenizer_config.json
Special tokens file saved in ./model-QA-wangchanberta/special_tokens_map.json


('./model-QA-wangchanberta/tokenizer_config.json',
 './model-QA-wangchanberta/special_tokens_map.json',
 './model-QA-wangchanberta/sentencepiece.bpe.model',
 './model-QA-wangchanberta/added_tokens.json',
 './model-QA-wangchanberta/tokenizer.json')

In [34]:
from transformers import pipeline
qa_model = pipeline(model="/content/model-QA-wangchanberta", 
                    tokenizer="/content/model-QA-wangchanberta", 
                    task="question-answering")

loading configuration file /content/model-QA-wangchanberta/config.json
Model config CamembertConfig {
  "_name_or_path": "airesearch/wangchanberta-base-att-spm-uncased",
  "architectures": [
    "CamembertForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "camembert",
  "num_attention_head": 12,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.11.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 25005
}

loading configuration file /content/model-QA-wangchanberta/config.json
Model config CamembertConfig {
  "_name_or_path

#Evaluate

In [ ]:
small_eval_set = datasets["validation"].select(range(74))
trained_checkpoint = "airesearch/wangchanberta-base-att-spm-uncased"

tokenizer = AutoTokenizer.from_pretrained(trained_checkpoint)
eval_set = small_eval_set.map(
    preprocess_validation_examples,
    batched=True,
    remove_columns= datasets["validation"].column_names,
)

In [ ]:
import torch
from transformers import AutoModelForQuestionAnswering

eval_set_for_model = eval_set.remove_columns(["example_id", "offset_mapping"])
eval_set_for_model.set_format("torch")

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
batch = {k: eval_set_for_model[k].to(device) for k in eval_set_for_model.column_names}
trained_model = AutoModelForQuestionAnswering.from_pretrained(trained_checkpoint).to(
    device
)

with torch.no_grad():
    outputs = trained_model(**batch)

In [37]:
start_logits = outputs.start_logits.cpu().numpy()
end_logits = outputs.end_logits.cpu().numpy()

In [38]:
import collections

example_to_features = collections.defaultdict(list)
for idx, feature in enumerate(eval_set):
    example_to_features[feature["example_id"]].append(idx)

In [41]:
import numpy as np

n_best = 20
max_answer_length = 30
predicted_answers = []

for example in small_eval_set:
    example_id = example["question_id"]
    context = example["context"]
    answers = []

    for feature_index in example_to_features[example_id]:
        start_logit = start_logits[feature_index]
        end_logit = end_logits[feature_index]
        offsets = eval_set["offset_mapping"][feature_index]

        start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
        end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
        for start_index in start_indexes:
            for end_index in end_indexes:
                # Skip answers that are not fully in the context
                if offsets[start_index] is None or offsets[end_index] is None:
                    continue
                # Skip answers with a length that is either < 0 or > max_answer_length.
                if (
                    end_index < start_index
                    or end_index - start_index + 1 > max_answer_length
                ):
                    continue

                answers.append(
                    {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                )

    best_answer = max(answers, key=lambda x: x["logit_score"])
    predicted_answers.append({"id": str(example_id), "prediction_text": best_answer["text"]})

In [42]:
import evaluate
metric = evaluate.load("squad")

In [43]:
theoretical_answers = [
    {"id": str(ex["question_id"]), "answers": { "answer_start": ex["answers"]["answer_begin_position"], "text" : ex["answers"]["answer"]}} for ex in small_eval_set]

In [44]:
print(predicted_answers)
print(theoretical_answers)

[{'id': '3', 'prediction_text': 'ไม่ต่ำกว่าพันล้านหยวน และเขาก็เป็นทายาทเพียงคนเดียวของตระกูล แต่เพราะเฝ'}, {'id': '5', 'prediction_text': 'ล่อนออกเป็นแว่นๆได้และมีเนื้อภายในเป็นสีเหลือง ใบนั้นจะเป็นใบประกอบแบบขนนก ดอกมีขนาดเล็กที่รวมกัน'}, {'id': '6', 'prediction_text': '</do'}, {'id': '8', 'prediction_text': 'ปกครองสเปนต่อจากพระเจ้าเฟร์นัน'}, {'id': '9', 'prediction_text': 'kok Film Festival 2007 และ Vancou'}, {'id': '10', 'prediction_text': 'ศึษาการเดินทาง การเดินทาง. รถโดยสารประจำทางที่ผ่าน ได้แก่ 2,3,5,6,7'}, {'id': '12', 'prediction_text': 'พากย์เสียงให้กับภาพยนตร์แอนิเมชันเรื่อง Clou'}, {'id': '13', 'prediction_text': 'าพักตรพิมล เสกสมรสกับหม่อมเจ้าประสพสุข ศุขสวัสดิ พระโอรสในพระเจ้าบรมวงศ์เธอ พระองค์เจ้าศุขสวัสดี กรมหลวงอดิศร'}, {'id': '14', 'prediction_text': 'เสาวนีย์ของ จักรพรรดินีนาถเก็มเม แต่ก็มีการย้ายไปที่อื่นอีก คือ คุ'}, {'id': '15', 'prediction_text': 'พัฒนาขึ้นมาเป็นรูปร่างซึ่งประกอบกันด้วยอาคารสำนักงานที่อยู่ส่วนล่างครอบคลุมชั้น 1-2 และชั้นที่ 3 ถึง 9'}, {'id': '16'

In [51]:
from tqdm.auto import tqdm

def compute_metrics(start_logits, end_logits, features, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["question_id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": str(example_id), "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [
    {"id": str(ex["question_id"]), "answers": {"text" : ex["answers"]["answer"], "answer_start": ex["answers"]["answer_begin_position"]}} for ex in small_eval_set]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

In [53]:
predictions, _, _ = trainer.predict(validation_dataset)
start_logits, end_logits = predictions
compute_metrics(start_logits, end_logits, validation_dataset, datasets["validation"])

The following columns in the test set  don't have a corresponding argument in `CamembertForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Prediction *****
  Num examples = 320
  Batch size = 16


  0%|          | 0/74 [00:00<?, ?it/s]

{'exact_match': 44.5945945945946, 'f1': 56.91832588891411}

#Implement

In [55]:
que = 'ดึกแล้วคุณขา เป็นภาพยนตร์แนวใด'
text ='''ดึกแล้วคุณขา ดึกแล้วคุณขา (Bangkok Time) เป็นภาพยนตร์ไทยที่กำกับโดยสันติ แต้พานิช ออกฉายในปี พ.ศ. 2550 
        โดยฉายแบบจำกัดโรงที่โรงภาพยนตร์ลิโด้ ดึกแล้วคุณขา เป็นภาพยนตร์รัก โรแมนติก มีเนื้อหาเกี่ยวกับรักสามเส้า ของชายสองหญิงหนึ่ง
        ที่เป็นคนทำงานกลางคืน คือ พ่อค้าขายของปลอมริมถนนสีลม (รับบทโดย อรรถพร ธีมากร) ชายหนุ่มบริการทางโทรศัพท์ (รับบทโดย อนันดา เอเวอริ่งแฮม) 
        และนางพยาบาลที่ต้องเข้าเวรกะดึก (รับบทโดย ดุสิตา อนุชิตชาญชัย) ภาพยนตร์เรื่องนี้เข้าฉายใน Bangkok Film Festival 2007 และ 
        Vancouver International Film Festival 2007 ในสาขา Dragon and Tiger Award'''
qa_model(question=que, context = text)

{'score': 0.12456362694501877, 'start': 177, 'end': 186, 'answer': ' โรแมนติก'}

#Test On Data

In [68]:
from datasets import load_dataset
test_data1 = load_dataset('csv', data_files='/content/drive/MyDrive/NLP III/answers.csv')

  0%|          | 0/1 [00:00<?, ?it/s]

In [69]:
test_data1

DatasetDict({
    train: Dataset({
        features: ['article_id', 'question_id', 'text', 'answer_start', 'answer_end', 'question', 'context'],
        num_rows: 50
    })
})

In [76]:
test_dataset = test_data1["train"].map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=test_data1["train"].column_names,
)

##Evaluate